In [ ]:
import pandas as pd # for reading csv
import matplotlib as mpl
import matplotlib.pyplot as plt # for plotting
import numpy as np # for higher degree regression
import pwlf # for bi-segmented linear regression
from sklearn.metrics import r2_score # for r2_scores
from matplotlib.font_manager import FontProperties
plt.rcParams.update({'xtick.labelsize': 18, 'ytick.labelsize': 18}) # big ticks
mpl.rcParams['font.family'] = 'Times New Roman'

In [ ]:
# function for fitting regression models
def fit_reg(X, Y, degree):
    coeffs = np.polyfit(X, Y, degree)
    func = np.poly1d(coeffs)
    return func

# function for fitting segmented linear regression models
def split_reg(X, Y, n):
    model = pwlf.PiecewiseLinFit(X, Y)
    bps = model.fit(n)
    return model, bps[1:-1]

# function to remove outliers
def std_filter(df, type):
    filtered_rows = []
    days = df['day'].unique()
    for day in days:
        group = df[df['day'] == day]
        mean = group[type].mean()
        std = group[type].std()
        print(f"[{type}] Day: {day}, Mean: {mean:.3f}, Std: {std:.3f}")
        filtered_group = group[(group[type] >= mean - std) & (group[type] <= mean + std)]
        filtered_rows.append(filtered_group)
    filtered_df = pd.concat(filtered_rows, ignore_index=True)
    filtered_df.to_csv(f"used_{type}.csv", index=False)
    return filtered_df

In [ ]:
con_df = pd.read_csv("test1/ctrl.csv") # reading control data
spd_df = pd.read_csv("test1/spd.csv") # reading spd data

# ignoring first days
lower_day_limit = 6 # days to ignore
con_df = con_df[con_df.day > lower_day_limit]
spd_df = spd_df[spd_df.day > lower_day_limit]

# removing outliers
con_filtered = std_filter(con_df, 'con')
spd_filetered = std_filter(spd_df, 'spd')

# converting into arrays for easy use
con_weights = con_filtered['con'].to_numpy()
con_days = con_filtered['day'].to_numpy()
spd_weights = spd_filetered['spd'].to_numpy()
spd_days = spd_filetered['day'].to_numpy()

In [ ]:
degree = 2 # degree for higher order regression
n = 2 # number of segments for linear regression


plot_toggle = 0
# 0: bi segmented linear regression
# 1: higher degree regression

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

# scatter plot
ax.scatter(con_weights, con_days, marker='o', s=25, facecolor='white', edgecolor='red', label='Con')
ax.scatter(spd_weights, spd_days, marker='s', s=25, facecolor='white', edgecolor='green', label='Spd')

# bi segmneted linear regression
if plot_toggle == 0:
    # fitting modlels
    con_model, con_bp = split_reg(con_weights, con_days, n)
    spd_model, spd_bp = split_reg(spd_weights, spd_days, n)
    # calculting critical weight ttp
    con_ttp = con_model.predict(con_bp)
    spd_ttp = spd_model.predict(spd_bp)
    # printing important results
    print(f"Control breakpoint(s): {con_bp}")
    print(f"Control ttp(s): {(con_ttp - 6)*24}")
    print(f"Control r2: {r2_score(con_days, con_model.predict(con_weights))}")
    print("\n")
    print(f"Spermidine breakpoint(s): {spd_bp}")
    print(f"Spermidine ttp(s): {(spd_ttp-6)*24}")
    print(f"Spermidine r2: {r2_score(spd_days, spd_model.predict(spd_weights))}")
    # plotting regression lines
    ax.plot(np.linspace(min(con_weights), max(con_weights), 100), con_model.predict(np.linspace(min(con_weights), max(con_weights), 100)), color='red')
    ax.plot(np.linspace(min(spd_weights), max(spd_weights), 100), spd_model.predict(np.linspace(min(spd_weights), max(spd_weights), 100)), color='green')
    # indicating breakpoints
    ax.scatter(con_bp, con_ttp-0.1, marker='^', color='red')
    ax.scatter(spd_bp, spd_ttp-0.1, marker='^', color='green')
    # indicating ttp
    for con_y, spd_y in zip(con_ttp, spd_ttp):
        ax.axhline(con_y, color='red', linewidth=1, linestyle='--')
        ax.axhline(spd_y, color='green', linewidth=1, linestyle='--')
    
    font = FontProperties(weight='bold', size=15)
    ax.legend(prop=font, frameon=False)
elif plot_toggle == 1:
    # fitting model
    con_model = fit_reg(con_weights, con_days, degree)
    spd_model = fit_reg(spd_weights, spd_days, degree)
    # printing r2
    print(f"Control r2: {r2_score(con_days, con_model(con_weights))}")
    print(f"Spermidine r2: {r2_score(spd_days, spd_model(spd_weights))}")
    # plotting curve
    ax.plot(np.linspace(min(con_weights), max(con_weights), 200), con_model(np.linspace(min(con_weights), max(con_weights), 200)), color='red')
    ax.plot(np.linspace(min(spd_weights), max(spd_weights), 200), spd_model(np.linspace(min(spd_weights), max(spd_weights), 200)), color='green')
    

# plot modifications
ax.set_xlim(left=0)
# ax.set_ylabel("Time to pupariation (hrs)", fontsize=18, fontweight='bold')
# ax.set_xlabel("Body weight (g)", fontsize=18, fontweight='bold')
# changing from days to hours after 6 days
max_days = max(max(con_days), max(spd_days))
ytick_positions = list(range(7, max_days + 1))
ytick_labels = [str((pos - 6) * 24) for pos in ytick_positions]
ax.set_yticks(ytick_positions)
ax.set_yticklabels(ytick_labels, fontweight='bold')
x_ticks = ax.get_xticks()
ax.set_xticks(x_ticks)
ax.set_xticklabels(x_ticks, fontweight='bold')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

# saving and displaying figure
plt.savefig("temp.jpg", bbox_inches='tight')
plt.show()